In [1]:
#Importing required libraries
import pandas as pd
import os
import re
import random
import itertools
import statistics

In [2]:
# List of folder paths
folder_paths = ['/Users/gb4818/OneDrive - Imperial College London/Rev-res-in/dispersion/to-move/100_results_w50_3dp',
                '/Users/gb4818/OneDrive - Imperial College London/Rev-res-in/dispersion/to-move/100_results_w50_4dp',
                '/Users/gb4818/OneDrive - Imperial College London/Rev-res-in/dispersion/to-move/100_results_w50_5dp',
                '/Users/gb4818/OneDrive - Imperial College London/Rev-res-in/dispersion/to-move/100_results_w50_6dp',
                
                '/Users/gb4818/OneDrive - Imperial College London/Rev-res-in/dispersion/to-move/100_results_w50_8dp',
                '/Users/gb4818/OneDrive - Imperial College London/Rev-res-in/dispersion/to-move/100_results_w50_10dp',
                '/Users/gb4818/OneDrive - Imperial College London/Rev-res-in/dispersion/to-move/100_results_w50_20dp',
                '/Users/gb4818/OneDrive - Imperial College London/Rev-res-in/dispersion/to-move/100_results_w50_25dp',
                
                '/Users/gb4818/OneDrive - Imperial College London/Rev-res-in/dispersion/to-move/100_results_w50_30dp',
                '/Users/gb4818/OneDrive - Imperial College London/Rev-res-in/dispersion/to-move/100_results_w50_35dp',
                '/Users/gb4818/OneDrive - Imperial College London/Rev-res-in/dispersion/to-move/100_results_w50_40dp',
                '/Users/gb4818/OneDrive - Imperial College London/Rev-res-in/dispersion/to-move/100_results_w50_50dp'
                ]

# Define folder labels
folder_labels = [3, 4, 5, 6, 8, 10, 20, 25, 30,  35, 40, 50]

The following code is to understand what is the number maximum and minimum of pairs


In [3]:
# Initialize an empty list to store the final results
dfs = []

# Loop through each folder path
for i, folder_path in enumerate(folder_paths):
    
    # Loop through each file in the folder
    for filename in os.listdir(folder_path):
        match = re.match(r'REvoSim_individuals_data_.*\.txt', filename)
        if match:
            file_path = os.path.join(folder_path, filename)

            # Read the DataFrame from the file
            df = pd.read_csv(file_path, sep=',', header=0, skiprows=12)
            genomes = df['Genome']
            num_pairs = len(genomes) // 2
            
            # Append the results to the final list
            dfs.append({
                'folder_name': folder_labels[i], 
                'file_name': filename,
                'num_pairs': num_pairs
            })

# Create a DataFrame from the results list
final_df = pd.DataFrame(dfs)

In [12]:
pairs_with_50 = final_df['num_pairs'].value_counts()[50]
len(final_df)
print(pairs_with_50/len(final_df)*100)


4.788418708240535


The following code is compouting only a small number of the possible pairs possibles.
Here at this link you can find an explanation of the following code:

https://www.notion.so/Genetic-Diversity-b4c384b7ba6e47e68a34148f50e9b530 

In [7]:
# Creating a function to create and store diversity values and return them in a table, 
# from this table we will calculate stats for each sample
def diversity_table(genomes, num_pairs):
    # Select random pairs of indices
    random_indices = random.sample(range(len(genomes)), 2 * num_pairs)

    # Initialize a counter for differences
    differences_list = []

    # Compare strings in random pairs
    for i in range(0, len(random_indices), 2):
        index1, index2 = random_indices[i], random_indices[i + 1]
        string1, string2 = genomes.iloc[index1], genomes.iloc[index2]

        # Compare strings and count differences
        differences_count = sum(c1 != c2 for c1, c2 in zip(string1, string2))

        # Append to the list
        differences_list.append((differences_count))

    return differences_list

I will try now to create a function that will look into all pairs. \
This involves iterating through all possible pairs of indices and calculating the number of differences for each pair. Here's a function that does this:

In [8]:
def diversity_table_all_pairs(genomes):
    # Initialize a list to store the number of differences for each pair
    differences_list = []
    
    # Get all possible pairs of indices
    all_pairs = itertools.combinations(range(len(genomes)), 2)
    
    # Compare genomes for each pair
    for index1, index2 in all_pairs:
        string1, string2 = genomes.iloc[index1], genomes.iloc[index2]
        
        # Count differences
        differences_count = sum(c1 != c2 for c1, c2 in zip(string1, string2))
        
        # Append the number of differences to the list
        differences_list.append(differences_count)
    
    return differences_list


Now we will try to test one and the other and see the differences, we are only going to takea sample so we do nto overwhelm the CPU with a very big task

In [10]:
# get a sample of genomes from a simulation
sample = pd.read_csv('/Users/gb4818/Library/CloudStorage/OneDrive-ImperialCollegeLondon/Rev-res-in/dispersion/to-move/REvoSim_individuals_data_envw50_3dp_ajTEST.txt', sep=',', header=0, skiprows=12,  usecols=[0])
genomes = sample['Genome']

# Calculate genetic diversity and store in the list
diversity_small = diversity_table(genomes, len(genomes) // 2)
diversity_all = diversity_table_all_pairs(genomes)

# Calculate the mean diversity for both lists
diversity_small_mean = statistics.mean(diversity_small)
diversity_all_mean = statistics.mean(diversity_all)

print('Mean diversity for small sample:', diversity_small_mean, 'for', len(genomes) // 2, 'pairs' )  
print('Mean diversity for all pairs:', diversity_all_mean, 'for', len(genomes) * (len(genomes) - 1) // 2, 'pairs' )
print('There is a difference of', diversity_all_mean - diversity_small_mean, 'between the two means')
print('The difference is', (diversity_all_mean - diversity_small_mean) / diversity_all_mean * 100, 'percent of the mean diversity for all pairs')


Mean diversity for small sample: 3.3333333333333335 for 3 pairs
Mean diversity for all pairs: 3.8095238095238093 for 21 pairs
There is a difference of 0.47619047619047583 between the two means
The difference is 12.499999999999991 percent of the mean diversity for all pairs
